## Quest
- 머신러닝을 이용한 감성분석
- 영화 댓글 학습(0:부정, 1:긍정)
- 타 사이트 댓글 이용 성능 확인

In [2]:
%pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=05ea051e063b66d6d6f13f4d18a01700dce4567d7cdf4db781e134245d34ee9d
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
import wget

In [5]:
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

'ratings_train.txt'

In [6]:
import pandas as pd
df_naver_movie_comments = pd.read_csv('./ratings_train.txt', sep='\t')
df_naver_movie_comments[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
df_naver_movie_comments = df_naver_movie_comments[:1000]

In [8]:
df_nagative = df_naver_movie_comments.loc[df_naver_movie_comments['label'] == 0]
df_nagative

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
...,...,...,...
993,9673323,마이클베이 최근작을 볼때마다 느낀다. 블록버스터 액션장르라도 탄탄한 시나리오는 영화...,0
994,8121444,0점은없나?1점은 너무 후하네,0
995,10275296,미달이는.. 연예계를 바라보는 태도를 고쳐야 한다.,0
996,6473973,파괴된 관객들.,0


In [9]:
df_positive = df_naver_movie_comments.loc[df_naver_movie_comments['label'] == 1]
df_positive

,id,document,label
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1
10,9008700,걍인피니트가짱이다.진짜짱이다♥,1
...,...,...,...
990,474835,마지막이 좀황당했지만 좋은영화,1
991,3279841,우연히 tv를 돌려보다가 사로잡혔다. 최근 10년간 본 코미디 영화 중 최고,1
992,8407277,연 기 굿 화이팅해요누나,1
997,8417568,알콜중독자들 덕에 평점이 낮은가?,1


In [28]:
negative_list = list(df_nagative['document'])

In [27]:
positive_list = list(df_positive['document'])

In [12]:
comments_list = negative_list + positive_list

### 불용어 처리

In [21]:
from mecab import MeCab
mecab = MeCab()

# 품사 달기
mecab.pos(comments_list[2])

[('교도소', 'NNG'),
 ('이야기', 'NNG'),
 ('구먼', 'VCP+EF'),
 ('.', 'SF'),
 ('.', 'SY'),
 ('솔직히', 'MAG'),
 ('재미', 'NNG'),
 ('는', 'JX'),
 ('없', 'VA'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('.', 'SY'),
 ('평점', 'NNG'),
 ('조정', 'NNG')]

In [31]:
# 불용어 리스트
stopwords = ['는','은','이','가','.']

# 품사 달기 및 불용어 제거
def remove_stopwords(comments_list):
    pos_result = mecab.pos(comments_list)  # 형태소 분석 결과 받아오기

    # 불용어 제거된 단어들 저장할 리스트 초기화
    filtered_words = []

    for word, pos in pos_result:
        if word not in stopwords:  # 단어가 불용어가 아닌 경우에만 추가
            filtered_words.append((word, pos))

    return filtered_words

# 예시 문장에 대해 품사 달기 및 불용어 제거하기
sentence = comments_list[2]  # 분석할 문장 입력

filtered_sentence = remove_stopwords(sentence)
print(filtered_sentence)

[('교도소', 'NNG'), ('이야기', 'NNG'), ('구먼', 'VCP+EF'), ('솔직히', 'MAG'), ('재미', 'NNG'), ('없', 'VA'), ('다', 'EF'), ('평점', 'NNG'), ('조정', 'NNG')]


### 형태소 분석기를 사용하여 품사 분류

In [14]:
!pip install python-mecab-ko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 26.8 MB/s eta 0:00:00


In [40]:
# 불용어 리스트
stopwords = ['는','은','이','가','.']

# 품사 달기 및 불용어 제거
def remove_stopwords(comments_list):
  morphs_result = mecab.morphs(comments_list)  # 형태소 분석 결과 받아오기

  # 불용어 제거된 단어들 저장할 리스트 초기화
  filtered_words = []

  for word in morphs_result:
    if word not in stopwords:  # 단어가 불용어가 아닌 경우에만 추가
      filtered_words.append((word))
  return filtered_words

tokenized_comments = list()
for comment in comments_list:
    comment_morphs = remove_stopwords(comment)
    tokenized_comments.append(' '.join(comment_morphs))
print(tokenized_comments)

['아 더 빙 진짜 짜증 나 네요 목소리', '너무 재 밓었다그래서보는것을추천한다', '교도소 이야기 구먼 솔직히 재미 없 다 평점 조정', '막 걸음마 뗀 3 세 부터 초등 학교 1 학년 생 인 8 살 용 영화 ㅋㅋㅋ .. 별반 개 도 아까움', '원작 의 긴장감 을 제대로 살려 내 지 못했 다', '별 반개 도 아깝 다 욕 나온다 이응경 길용우 연기 생활 몇 년 인지 정말 발 로 해도 그것 보단 낫 겟 다 납치 감금 만 반복 반복 드라마 가족 도 없 다 연기 못 하 사람 만 모엿 네', '울 면서 손들 고 횡단 보도 건널 때 뛰쳐나올 뻔 이범수 연기 드럽 게 못해', '취향 존중 한다지만 진짜 내생 에 극장 에서 본 영화 중 가장 노 잼 노 감동 임 스토리 도 어 거지 고 감동 도 어거지', '굿바이 레닌 표절 인 것 이해 하 는데 왜 뒤 로 갈수록 재미없 어 지 냐', '재미없 다 지루 하 고 같 음식 영화 인데 도 바베트 의 만찬 하 고 넘 차이 남 ... 바베트 의 만찬 이야기 도 있 고 음식 보 재미 도 있 는데 ; 이건 볼 게 없 다 음식 도 별로 안 나오 고 , 핀란드 풍경 라도 구경 할 랫 는데 그것 도 별로 안 나옴 ㅡㅡ', '주제 좋 은데 중반 부터 지루 하 다', '다 짤랐 을 꺼 야 그래서 납득 할 수 없 었 던 거 야 그럴 꺼 야 꼭 그랬 던 걸 꺼 야', '카밀라벨 발연기', '졸 쓰레기 진부 하 고 말 도 안 됌ㅋㅋ 아 시간 아까워', '1 % 라도 기대 했 던 내 죄인 입니다 죄인 입니다 ...', '키이라 나이틀리 연기 하 고자 했 던 건 대체 정신 장애 일까 틱 장애 일까', '포스터 있 어 보이 는데 관객 114 명 네', "' 다 알 바 생 인가 내용 도 없 고 무서운 거 도 없 고 웃긴 거 도 하나 도 없 음 완전 별 싱거운 영화 ㅇ ㅇ내ㅇ시간 넘 아까 움 완전 낚 임", '평점 에 속지 마시 길 시간 낭비 돈 낭비 임', '리얼리티 뛰어나 긴 한데 큰 공감 안 간다 이민기 캐릭터 정신 의 학상 분노 조절 장애 초기 증상

### 벡터 토큰화(단어 숫자화)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [43]:
countVectorizer = CountVectorizer()
countVectorizer.fit(tokenized_comments)

CountVectorizer()

In [44]:
countVectorizer.vocabulary_

{'진짜': 2500,
 '짜증': 2510,
 '네요': 546,
 '목소리': 1025,
 '너무': 536,
 '밓었다그래서보는것을추천한다': 1128,
 '교도소': 300,
 '이야기': 2156,
 '구먼': 311,
 '솔직히': 1511,
 '재미': 2274,
 '평점': 2728,
 '조정': 2385,
 '걸음마': 214,
 '부터': 1273,
 '초등': 2560,
 '학교': 2782,
 '학년': 2783,
 '영화': 1915,
 'ㅋㅋㅋ': 78,
 '별반': 1213,
 '아까움': 1666,
 '원작': 2026,
 '긴장감': 415,
 '제대로': 2358,
 '살려': 1374,
 '못했': 1043,
 '반개': 1157,
 '아깝': 1670,
 '나온다': 484,
 '이응경': 2160,
 '길용우': 418,
 '연기': 1880,
 '생활': 1420,
 '인지': 2194,
 '정말': 2331,
 '해도': 2829,
 '그것': 339,
 '보단': 1219,
 '납치': 521,
 '감금': 141,
 '반복': 1159,
 '드라마': 775,
 '가족': 122,
 '사람': 1350,
 '모엿': 1019,
 '면서': 997,
 '손들': 1506,
 '횡단': 2916,
 '보도': 1220,
 '건널': 203,
 '뛰쳐나올': 828,
 '이범수': 2143,
 '드럽': 780,
 '못해': 1042,
 '취향': 2594,
 '존중': 2392,
 '한다지만': 2797,
 '내생': 530,
 '극장': 377,
 '에서': 1844,
 '가장': 115,
 '감동': 144,
 '스토리': 1558,
 '거지': 196,
 '어거지': 1770,
 '굿바이': 327,
 '레닌': 854,
 '표절': 2744,
 '이해': 2173,
 '는데': 599,
 '갈수록': 138,
 '재미없': 2275,
 '지루': 2473,
 '음식': 2095,
 '인데': 2186,
 '바베트

In [45]:
tfidfVectorizer = TfidfVectorizer(use_idf=True)
tfidfVectorizer.fit(tokenized_comments)

TfidfVectorizer()

In [46]:
tfidfVectorizer.idf_

array([7.2156076 , 4.73070095, 6.29931687, ..., 7.2156076 , 6.81014249,
       6.11699531])

In [48]:
features = tfidfVectorizer.transform(tokenized_comments)

### 목표변수 생성

In [50]:
target = ['긍정'] * len(positive_list) + ['부정'] * len(negative_list)
target

['긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',

### 머신러닝 학습

In [51]:
from sklearn.svm import SVC

In [52]:
model = SVC()
model.fit(features.toarray(), target)

SVC()

### 머신러닝을 통한 에측
- 다음사이트 '영화' 댓글을 이용해 예측

In [53]:
comment = '놀란의 영화는 정말 미쳤다는 말밖에 안 나온다. 광복절을 맞이하여 이런 영화를 선사해준 놀란에게 경의를 표합니다.'

In [54]:
tokenized_comment = mecab.morphs(comment)

In [55]:
vectorized_comment = tfidfVectorizer.transform([' '.join(tokenized_comment)]) # 2차원으로 넣어줌
vectorized_comment.toarray()  #벡터화

array([[0., 0., 0., ..., 0., 0., 0.]])

In [56]:
model.predict(vectorized_comment.toarray())

array(['부정'], dtype='<U2')